In [1]:
import psycopg2
import numpy as np
import pandas as pd

In [2]:
# DB Connect
conn = psycopg2.connect(
    host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port=5439,
    dbname='dev',
    user='awsuser',
    password='cremaoAdmin1234qwer!!'
)
cursor = conn.cursor()

### 1. 연별 합계 (sum)

In [41]:
# get data from DB
def get_data(col, date, df):
    query = f'select sum({col}) from m1.kt_bldg_xy_info where base_ym between {date}01 and {date}12' # 쿼리문
    cursor.execute(query) # 실행
    df = cursor.fetchall() # 출력
    df = df[0][0] # 값 꺼내기

# sum
def getSum(colName):
    for i in range(3):
        dfName = f'{colName}_202{i}'
        get_data('f_tot', '2020', dfName)
        print(colName + ':' + dfName)



getSum('f_tot') # 유동인구
getSum('pop') # 거주인구
getSum('wpop') # 직장인구

f_tot:f_tot_2020
f_tot:f_tot_2021
f_tot:f_tot_2022
pop:pop_2020
pop:pop_2021
pop:pop_2022
wpop:wpop_2020
wpop:wpop_2021
wpop:wpop_2022


### 2. 전년대비 증감율

In [ ]:
# 결과 데이터레임 생성
DF = f_tot['f_tot']

In [7]:
# 전년 대비 증감율 
def perYearIncrease(df_prior, df_current, df, col): # df_prior=전기, df_current=당기, df=전기 데이터프레임, col=추가할 컬럼명
    if (df_prior/df_current)*100 >= 120: # 120% 이상: 1
        df[col] = 1
    elif (df_prior/df_current)*100 < 120: # 120% 미만: 0
        df[col] = 0

In [10]:
# 실행
# 유동인구
perYearIncrease(f_tot_2020, f_tot_2021, DF, 'f_tot') # 2021년도 증감율
perYearIncrease(f_tot_2021, f_tot_2022, DF, 'f_tot') # 2022년도 증감율

# 거주인구
perYearIncrease(pop_2020, pop_2021, DF, 'pop') # 2021년도 증감율
perYearIncrease(pop_2021, pop_2022, DF, 'pop') # 2022년도 증감율

# 직장인구
perYearIncrease(wpop_2020, wpop_2021, DF, 'wpop') # 2021년도 증감율
perYearIncrease(wpop_2021, wpop_2022, DF, 'wpop') # 2022년도 증감율

In [ ]:
# 비율 파악 (0 대비 1의 비율)
# 유동인구
len(df[df['f_tot']==1]) / len(df[df['f_tot']==0])*100

# 거주인구
len(df[df['pop']==1]) / len(df[df['pop']==0])*100

# 직장인구
len(df[df['wpop']==1]) / len(df[df['wpop']==0])*100